# Import packages

In [1]:
# imported packages
from __future__ import print_function
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

%matplotlib inline
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import matplotlib.ticker as plticker
import bisect
import copy
import json
import csv
import ast
# from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import interp1d
import time

# Lists of colors for plots

c0 = (0.76, 0.76, 0.76)
c1 = (1.00, 0.18, 0.33);
c2 = (1.00, 0.23, 0.19);
c3 = (1.00, 0.58, 0.00);
c4 = (1.00, 0.80, 0.00);
c5 = (0.30, 0.85, 0.39);
c6 = (0.35, 0.78, 0.98);
c7 = (0.20, 0.67, 0.86);
c8 = (0.00, 0.48, 1.00);
c9 = (0.35, 0.34, 0.84);
c10 = (0.00, 0.31, 0.57);
c11 = (0.12, 0.29, 0.69);
c12 = (0.17, 0.17, 0.42);
c13 = (1.00, 1.00, 1.00);
c14 = (0.77, 0.04, 0.00);

ios_colors=[c5, c1, c3, c7, c2, c6, c4, c9, c8, c10, c14, c12]

def to_rgb_frac(vect):
    return (float(vect[0])/255, float(vect[1])/255, float(vect[2])/255)

color_list=list(map(to_rgb_frac, [(166,206,227),(178,223,138),(51,160,44),(31,120,180)]))

# Lists of colors for plots

def to_rgb_frac(vect):
    return (float(vect[0])/255, float(vect[1])/255, float(vect[2])/255)

rb_div_5=list(map(to_rgb_frac, [(202,0,32),
(244,165,130),
(247,247,247),
(146,197,222),
(5,113,176)]))

qualitative_10=list(map(to_rgb_frac, [   
(99,99,99),
(178,223,138),
(51,160,44),
(251,154,153),
(227,26,28),
(253,191,111),
(255,127,0),
(202,178, 214),
(106,61,154),
(166,206,227),    
(31,120,180),  
(189,189,189),
]))

# Reading in `dfe_dict`

In [2]:
# DFE with delta function effects
dfe_dict={
 0.3: {'LoF': 10**-5}}
s=0.3

# Functions to randomly sample from ``dfe_dict`` 

In [3]:
# Function to randomly sample from a (mutation, fitness) pair. 
mutations=[]
fitnesses=[]
rates=[]
for k, v in dfe_dict.items():
    for kk, vv in v.items():
        if kk!= 'unknown':
            mutations.append(kk)
            fitnesses.append(k)
            rates.append(vv)

Ub=sum(rates)

normalized_cumulative=0.0
cumulative_list=[]
mutation_fitness_generator={}
            
for i, (m, s, r) in enumerate(zip(mutations, fitnesses, rates)):
    normalized_cumulative=normalized_cumulative+r/Ub
    cumulative_list.append(normalized_cumulative)
    mutation_fitness_generator[i]=(m, s)
    
    
def map_rand_to_index(rand, cumulative_list):
    for i, c in enumerate(cumulative_list):
        if rand < c:
            break
    return i

    
def random_mut_fit_generator(cumulative_list, mutation_fitness_generator_dict):
    eta=random.random()
    ind=map_rand_to_index(eta, cumulative_list)
    return mutation_fitness_generator_dict[ind]


# Useful functions

This is modified to be quasi-deterministic

In [4]:
# mutate_establish: takes (genotype=list of (mut IDs, s) tuples, abundance) and generates a set of new mutant (genotype, abundance) tuples AFTER establishment.

from decimal import Decimal
from scipy import stats
import numpy as np

def mutate_establish(g, current_mut_ID, mutation_rate, mean_fitness, t, dt):
    abundance=g[1]
    current_genotype=g[0]
    genotype_fitness=sum(list(map(lambda x: x[2], current_genotype)))
    genotype_advantage=genotype_fitness-mean_fitness
    list_of_mutations=list(map(lambda x: x[1], current_genotype)) #e.g. ['IRA1', 'GPB2']
    number_new_mutants=np.random.poisson(mutation_rate*abundance)
#     print('mutation_rate*abundance', mutation_rate*abundance)
    new_genotypes_created=[]
    
    for m in range(number_new_mutants):
        mut_id=current_mut_ID+m+1
        
        #create random mutation in form (mut_type, s)
        new_mut=random_mut_fit_generator(cumulative_list, mutation_fitness_generator)
        new_mut_type=new_mut[0]
        new_mut_fitness=new_mut[1]
        
# #         full information
#         new_genotype=current_genotype+[(mut_id, new_mut_type, new_mut_fitness, t*dt)]
#         masking information
        new_genotype=current_genotype+[(mut_id, new_mut_type, new_mut_fitness)]
        new_entry=(new_genotype, 1)                
        new_genotypes_created.append(new_entry)
            
    return new_genotypes_created


# mutate_population
"returns the new barcode dictionary after mutation and the mut_id of latest mutation"

def mutate_pop(barcodes, mut_ID, mut_rate, mean_fitness, t, dt):
    new_barcode_dict={}
    mutation_counter=mut_ID
    
    for bc, genotypes in barcodes.items():
        genotypes_after_mutation=genotypes
        for g in genotypes:

            new_mutant_genotypes=mutate_establish(g, mutation_counter, mut_rate, mean_fitness, t, dt)
            if len(new_mutant_genotypes)>0:
                mutation_counter=new_mutant_genotypes[-1][0][-1][0]
            genotypes_after_mutation=genotypes_after_mutation+new_mutant_genotypes

        new_barcode_dict[bc]=genotypes_after_mutation
    return (new_barcode_dict, mutation_counter)

# Population size

def population_size(barcodes):
    total_abundance=0
    for k, v in barcodes.items():
        genotypes=v
        for g in genotypes:
            abundance=g[1]
            total_abundance=total_abundance+abundance
    return total_abundance

# Samples from a weighted list
def weighted_choice(choices_list, number_of_samples):
    
    total = sum(w for c, w in choices_list)
    
    upto = 0
    cumulative_entry_map={}
    interval_list=[0]
    for c, w in choices_list:
        upto=upto+w
        cumulative_entry_map[upto]=c
        interval_list.append(upto)
        
    choices=[]
    
    for j in range(number_of_samples):
        
        x=np.random.uniform(0, total)
        i = bisect.bisect_left(interval_list,x)
        k=interval_list[i]
        choice=cumulative_entry_map[k]
        choices.append(choice)
    return choices

# Mean fitness: calculated population mean fitness

def mean_fitness(barcodes):
    total_abundance=0
    total_fitness=0.0
    for k, v in barcodes.items():
        genotypes=v
        for g in genotypes:
            mutations=g[0]
            abundance=g[1]
            total_abundance=total_abundance+abundance
            fitness=0.0
            for m in mutations:
                fitness=fitness+m[2]
            total_fitness=total_fitness+abundance*fitness
    mean_fitness=total_fitness/total_abundance
    return mean_fitness

# Selection on genotype. Takes a genotype and returns a new genotype whose frequecies have changed due to selection

def select(g, absolute_fitness, mean_fitness, t, dt, kappa):
    "absolute fitness is to allow actual expansion in absence of fitness diffs"
    abundance=g[1]
    mutations=g[0]
#     establishment_year = mutations[len(mutations)-1][3]
#     status = mutations[len(mutations)-1][4]
    fitness=0.0
    for m in mutations:
        fitness=fitness+m[2]
    lead=fitness-mean_fitness
    
    if abundance == 0:
        new_abundance_with_noise = 0
    else:
#         approximation
        new_abundance_with_noise=np.random.poisson(abundance*(1+lead*dt))

        
    return (mutations, new_abundance_with_noise)

# select_pop function performs selection on entire population of barcodes
def select_pop(barcodes, absolute_fitness, mean_fitness, t, dt, kappa):
    new_barcode_dict={}
    for bc, genotypes in barcodes.items():
        genotypes_after_selection=[select(g, absolute_fitness, mean_fitness, t, dt, kappa) for g in genotypes]
        new_barcode_dict[bc]=genotypes_after_selection
    return new_barcode_dict

# Re-barcode function
"rebarcodes population by inserting defined number of new barcodes making composite barcodes each of which is represented in one cell"

def re_barcode(current_barcode_dict, number_of_new_barcodes, number_cells_to_be_barcoded):
    
    N=population_size(current_barcode_dict)

    new_barcodes={}
    genotype_frequencies=[]
    for bc, genotypes in current_barcode_dict.items():
        for g in genotypes:
            genotype_frequencies.append([(bc, g[0]), float(g[1])/N])
            
    genotype_choices=weighted_choice(genotype_frequencies, number_cells_to_be_barcoded)
    
    new_barcode_frequencies=[(j, 1) for j in range(number_of_new_barcodes)]
    new_barcode_choices=weighted_choice(new_barcode_frequencies, number_cells_to_be_barcoded)
    
    for new_bc, old_g in zip(new_barcode_choices, genotype_choices):
        old_bc=old_g[0]
        mutation_list=old_g[1]
        composite_bc=tuple([j for j in old_bc]+[new_bc])
        if composite_bc in new_barcodes:
            new_barcodes[composite_bc].append((mutation_list, 1))
        else:
            new_barcodes[composite_bc]=[(mutation_list, 1)]
    return new_barcodes

# Sample cells function
"A function that simply samples some number of the currently present genotypes"

def sample_cells(current_barcode_dict, sample_size):
    N=population_size(current_barcode_dict)
    new_barcode_dict={}
    for bc, genotypes in current_barcode_dict.items():
        new_genotypes_within_this_bc=[]
        for g in genotypes:

            mutation_list=g[0]
            current_abundance=g[1]
            mean_new_abundance=current_abundance*(float(sample_size)/N)

            if mean_new_abundance>300:
                new_abundance=round(np.random.poisson(mean_new_abundance))
            elif 0.0<mean_new_abundance<=300:
                new_abundance=round(np.random.poisson(mean_new_abundance))
            else:
                new_abundance=0

            new_genotype=(mutation_list, new_abundance)
            new_genotypes_within_this_bc.append(new_genotype)
        new_barcode_dict[bc]=new_genotypes_within_this_bc
    return new_barcode_dict

# Purge extinct genotypes and extinct barcodes
# def purge(barcodes, mean_fitness):
#     new_barcode_dict={}
#     for bc, genotypes in barcodes.items():
#         genotypes_after_purging=[]
#         for g in genotypes:
#             genotype_fitness=sum(list(map(lambda x: x[2], g[0])))
#             genotype_advantage=genotype_fitness-mean_fitness
            
#             if genotype_advantage<0 and g[1]<10:
#                 new_size=np.random.poisson(g[1])
#                 new_g=g[0]
                
#                 if new_size>0:
#                     genotypes_after_purging.append((new_g, new_size))
     
#             else:
#                 genotypes_after_purging.append(g)
#         if len(genotypes_after_purging)>0:
#             new_barcode_dict[bc]=genotypes_after_purging
#     return new_barcode_dict

def purge(barcodes, mean_fitness):
    new_barcode_dict={}
    for bc, genotypes in barcodes.items():
        genotypes_after_purging=[]
        for g in genotypes:
#             print('g', g)
            if g[1] != 0:
                genotypes_after_purging.append(g)
                
        if len(genotypes_after_purging)>0:
            new_barcode_dict[bc]=genotypes_after_purging
    return new_barcode_dict

# sequence_barcodes function: converts barcodes to read counts:
def sequence_barcodes(barcodes, read_trajectories_dict, depth):
    N=population_size(barcodes)
    new_read_trajectories_dict={}
    for bc in read_trajectories_dict.keys():
        if bc in barcodes:
            genos=barcodes[bc]
            frequency_of_bc=float(sum(g[1] for g in genos))/N
            expected_read_number=frequency_of_bc*depth
            if expected_read_number>300:
                read_number=int(np.random.normal(expected_read_number, (expected_read_number)**0.5))
            else:
                read_number=np.random.poisson(expected_read_number)
        else:
            read_number=0
        
            
        current_read_traj=read_trajectories_dict[bc]
        current_read_traj.append(read_number)
        
        new_read_trajectories_dict[bc]=current_read_traj
        
    return new_read_trajectories_dict

# remap keys function for writing dictionarie to file
def remap_keys(mapping):
    return [{'key':k, 'value': v} for k, v in mapping.items()]

def genotype_count(barcodes):
    g_count=0
    for bc, genotype in barcodes.items():
        g_count=g_count+len(genotype)
    return g_count

In [5]:
def reading_the_main_branch_sizes(barcodes, final_cancer_lineage):
    
#     print('final_cancer_lineage', final_cancer_lineage)
    
    list_of_clones = barcodes.values()    
     
    n = 0
    string_of_num=[0,0,0,0,0]
    for clone in list(list_of_clones)[0]:
        clone_ID = list(clone[0])
        clone_size = clone[1]
        
        not_main_branch = False
#         print('clone_ID', clone_ID)
        for mut_ID in clone_ID:
            if mut_ID not in final_cancer_lineage:
                not_main_branch = True
        if not_main_branch == False:            
            string_of_num[len(clone_ID)-1] = clone_size
            n += 1
#     print('\nstring_of_num', string_of_num, 'entries', n)
    
    return string_of_num

# Perform dynamics below

In [6]:
# Dynamics of growth

def main():
    
    age_of_control = {}
    age_of_cancer_onset = {}
    
    num_of_controls = 0
    num_gen_per_year = 10
    dt = 1/num_gen_per_year
    print('dt', dt)

    kappa = 1
    # number of generations
    T=num_gen_per_year*number_of_years

    Ub=0.0
    for s, m_u in dfe_dict.items():
        for m, u in m_u.items():
            Ub=Ub+u

    pop_size=10**5
    mutation_counter=0

    barcodes={}

    #pre-existing diploids
    # barcodes[(i,)]=[([(0,'WT', 0.0)],10**8), ([(0,'WT', 0.0),(0,'DIPLOID', 0.025)],10**6)]

    #no prexisting diploids


    # #insert second BC library into the pool with pre-existing diploids
    # barcodes=re_barcode(barcodes, number_of_second_barcodes, number_of_second_barcode_insertions)
    # print(barcodes)


    barcode_master_pool=barcodes

    lt=time.time()
    
    number_of_cases = 0
    number_of_80_100_cases =0
    number_of_60_80_cases = 0
    number_of_lessthan60_cases = 0

    for R in range(number_of_replicates):
        
        if R%1000 == 0:
            print('R', R)
            
        final_cancer_lineage = None
        generation=0
        K_hit_occurred = False
        barcodes = {}
#        mutation ID, mutation type, fitness, emergence time
        barcodes[(R,)]=[([(0,'WT', 0.0, 0)], pop_size)]

    #     for t in range(T+1):
        for t in range(T+1):
    #         print('generation ', t, ' ')

            xbar=mean_fitness(barcodes)
    #         set absolute fitness to zero
    #         absolute_fitness=math.log(1.0)
            absolute_fitness=0

            #mutate
            mut=mutate_pop(barcodes, mutation_counter, Ub*dt, xbar, t, dt)
            barcodes_after_mutation=mut[0]
            mutation_counter=mut[1]
#             print('mutation_counter', mutation_counter)

            #selection
            barcodes_after_selection=select_pop(barcodes_after_mutation, absolute_fitness, xbar, t, dt, kappa)
            #         print('select')

            #purging extinct genotypes
            
            barcodes=barcodes_after_selection

            for clone in barcodes[(R,)]:
                clone_ID = clone[0]
#                 print('clone_ID', clone_ID)
                if len(clone_ID) == K:
                    final_cancer_lineage = clone_ID
#                     print('clone_ID', clone_ID)
                    K_hit_occurred = True
                    

            if K_hit_occurred == True:
                
                string_of_num = reading_the_main_branch_sizes(barcodes, final_cancer_lineage)

                age_of_cancer_onset[R]=generation
                if 80*num_gen_per_year < generation < 100*num_gen_per_year and number_of_80_100_cases <= targeted_tree_num:
                    barcode_dict_filename='alternative_hypothesis//age_80_100//K_4_s_30p_10minus5_positive//clonal_tree_R_'+str(R)+'.txt'
                    number_of_80_100_cases += 1 
                    with open(barcode_dict_filename, 'w') as outfile:
                        json.dump(string_of_num, outfile)
                if 60*num_gen_per_year < generation < 80*num_gen_per_year and number_of_60_80_cases <= targeted_tree_num:
                    barcode_dict_filename='alternative_hypothesis//age_60_80//K_4_s_30p_10minus5_positive//clonal_tree_R_'+str(R)+'.txt'
                    number_of_60_80_cases += 1
                    with open(barcode_dict_filename, 'w') as outfile:
                        json.dump(string_of_num, outfile)
                if generation < 60*num_gen_per_year and number_of_lessthan60_cases <= targeted_tree_num:
                    barcode_dict_filename='alternative_hypothesis//age_lessthan_60//K_4_s_30p_10minus5_positive//clonal_tree_R_'+str(R)+'.txt'
                    number_of_lessthan60_cases += 1
                    with open(barcode_dict_filename, 'w') as outfile:
                        json.dump(string_of_num, outfile)
            #             print('write to file')

#                 print('generation,', generation)
                number_of_cases += 1
                break 
        
            else:
                barcodes=purge(barcodes, xbar)
        
                          
            generation=generation+1
            
        if number_of_60_80_cases >= targeted_tree_num and number_of_lessthan60_cases >= targeted_tree_num:
            if number_of_80_100_cases >= targeted_tree_num:
                break
        
        
          
#         if K_hit_occurred != True:
#             num_of_controls = num_of_controls + 1
#             if num_of_controls  < control_size:
#                 age_of_control[R]=generation
#                 barcode_dict_filename='testing_subdirectory_negative//DETERMINISTIC_s_15p_10minus5_negative_clonal_tree_R_'+str(R)+'.txt'
# #                 with open(barcode_dict_filename, 'w') as outfile:
# #                     json.dump(remap_keys(barcodes), outfile)
#             else:
#                 break
            
#     print('here')
#     if 60*num_gen_per_year < generation < 80*num_gen_per_year:
#         K_hit_gen_filename='multiclass_classifier_training//age_60_80//K_4_s_15p_10minus4_positive//event_IDs.txt'
#     with open(K_hit_gen_filename, 'w') as outfile:
#         json.dump(remap_keys(age_of_cancer_onset), outfile)
        
#     K_hit_gen_filename='testing_subdirectory_negative//DETERMINISTIC_k_hit_gen_s_15p_10minus5_control_ages.txt'
#     with open(K_hit_gen_filename, 'w') as outfile:
#         json.dump(remap_keys(age_of_control), outfile)
    
        
    
    return None

In [8]:
# hyper parameters
# control_size = 1000000
K=4+1
targeted_tree_num = 5000
number_of_years = 100 # this is the age of all controls
number_of_replicates = 1000000000 # number of total people: cancer + controls
        

if __name__ == '__main__':
    main()
    

dt 0.1
R 0
R 1000
R 2000
R 3000
R 4000
R 5000
R 6000
R 7000
R 8000
R 9000
R 10000
R 11000
R 12000
R 13000
R 14000
R 15000
R 16000
R 17000
R 18000
R 19000
R 20000
R 21000
R 22000
R 23000
R 24000
R 25000
R 26000
R 27000
R 28000
R 29000
R 30000
R 31000
R 32000
R 33000
R 34000
R 35000
R 36000
R 37000
R 38000
R 39000
R 40000
R 41000
R 42000
R 43000
R 44000
R 45000
R 46000
R 47000
R 48000
R 49000
R 50000
R 51000
R 52000
R 53000
R 54000
R 55000
R 56000
R 57000
R 58000
R 59000
R 60000
R 61000
R 62000
R 63000
R 64000
R 65000
R 66000
R 67000
R 68000
R 69000
R 70000
R 71000
R 72000
R 73000
R 74000
R 75000
R 76000
R 77000
R 78000
R 79000
R 80000
R 81000
R 82000
R 83000
R 84000
R 85000
R 86000
R 87000
R 88000
R 89000
R 90000
R 91000
R 92000
R 93000
R 94000
R 95000
R 96000
R 97000
R 98000
R 99000
R 100000
R 101000
R 102000
R 103000
R 104000
R 105000
R 106000
R 107000
R 108000
R 109000
R 110000
R 111000
R 112000
R 113000
R 114000
R 115000
R 116000
R 117000
R 118000
R 119000
R 120000
R 121000
R 122000

KeyboardInterrupt: 